# 2.b Linear Discriminant Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./train_MNIST_LDA.csv')

$X=\begin{pmatrix} x_1 \\ \vdots \\ x_n \end{pmatrix}$

In [3]:
Xpred = list(data.columns)
Xpred.remove('label')
Xcols = Xpred

n = data.shape[0]
p = len(Xpred)

In [4]:
X = data[Xcols].as_matrix()
y = list(data.label)

### Coefficient

In [5]:
K = [0, 2, 4]
nk = {}
mu = {}
sig = {}
for k in K:
    nk[k] = y.count(k)
    mu[k] = 0
    sig[k] = 0
for i in range(n):
    k = y[i]
    mu[k] = mu[k] + X[i]
for k in K:
    mu[k] = mu[k] / nk[k]
for i in range(n):
    k = y[i]
    sig[k] = sig[k] + np.outer(X[i] - mu[k], X[i] - mu[k])
sig_hat = sum(sig.values()) / (n - len(K))
# sig_hat is singular, use pinv instead
sig_inv = np.linalg.pinv(sig_hat)

### Test

In [6]:
test_data = pd.read_csv('./test_MNIST_LDA.csv', names=Xpred)
label_data = pd.read_csv('./label_MNIST_LDA.csv', header=None)

In [7]:
yk = {}
for k in K:
    yk[k] = np.dot(np.dot(test_data, sig_inv), mu[k])
    - np.dot(np.dot(mu[k].T, sig_inv), mu[k]) / 2
    + np.log(nk[k] / n)

In [8]:
delta = np.argmax(list(yk.values()), axis=0)
res = [K[k] for k in delta]

### Confusion Matrix

In [9]:
confusion_list = list(zip(list(label_data[0]), res))
cmatrix = [[confusion_list.count((k1, k2)) for k2 in K] for k1 in K]

In [10]:
pd.DataFrame([
    *[[*cmatrix[i], sum(cmatrix[i])] for i in range(3)],
    list(np.sum(cmatrix, axis=0)) + [np.sum(cmatrix)]],
    columns=[[''] * 4, ['Confusion Matrix'] * 4, ['True class'] * 4, [*K, 'Total']],
    index=[['Predicted class'] * 4, [*K, 'Total']]
)

Confusion Matrix                
                            True class                
                                     0    2    4 Total
Predicted class 0                  472    7    1   480
                2                   12  408    3   423
                4                    1   13  465   479
                Total              485  428  469  1382